In [ ]:
# case study 
# exp baseline artistic-toad  : powerful-crow type 2    6个比赛4个比基线高 curious-cattle type 1    6个比赛5个比基线高 
import pandas as pd 
import pickle


In [ ]:
baseline = "smooth-tuna"

wo = "quality-cougar"
EXP = [wo,baseline]


EVO = [ "cassava-leaf-disease-classification",
            "h-and-m-personalized-fashion-recommendations",
            "jigsaw-toxic-comment-classification-challenge",
            "leaf-classification",
            "tweet-sentiment-extraction",
            "us-patent-phrase-to-phrase-matching",
            "whale-categorization-playground",
            "learning-agency-lab-automated-essay-scoring-2",
            "aptos2019-blindness-detection",
            "kuzushiji-recognition",
            "herbarium-2020-fgvc7",
            "text-normalization-challenge-russian-language",
            "rsna-miccai-brain-tumor-radiogenomic-classification",
            "freesound-audio-tagging-2019",
            "mlsp-2013-birds",
            "spooky-author-identification",
            "hubmap-kidney-segmentation",]

In [ ]:
F = []

for exp in EXP:
    p0= f'/data/share_folder_local/amlt/{exp}/combined_logs/summary.pkl'
    with open(p0, 'rb') as f:
        summary  = pickle.load(f)
    for idx, competition in enumerate(summary.keys()):
        if competition.split(".")[-2] in EVO:
            data = summary[competition]
            print(f"Experiment: {exp}, Competition: {competition}")
            test_scores_df = pd.DataFrame.from_dict(data["test_scores"], orient="index", columns=["Test Score"])
            test_scores_df.index.name = "Loop"
            
            # valid_scores_dict = data["valid_scores"]
            # # 提取 ensemble 验证分数
            # ensemble_scores = {}
            # for loop_id, df in valid_scores_dict.items():
            #     if "ensemble" in df.index:
            #         ensemble_scores[loop_id] = df.loc["ensemble"].iloc[0]
            ensemble_scores = test_scores_df["Test Score"]
            #print(ensemble_scores)
            bronze_threshold = data["bronze_threshold"]
            silver_threshold = data["silver_threshold"]
            if ensemble_scores.size > 0:
                if bronze_threshold > silver_threshold:
                    F.append((exp, competition, ensemble_scores.min(), -1))
                else:
                    F.append((exp, competition, ensemble_scores.max(), 1))

In [ ]:
F = [
    item for item in F
    if item[1].endswith(('.1', '.2', '.3'))
]


In [ ]:
F

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np

data = F  # F 内部为 (exp, comp, score, direction)

def strip_version(name):
    if "." in name:
        return name.rsplit(".", 1)[0]
    return name

# 第一步：按 competition + experiment 聚合所有 score
comp2scores_raw = defaultdict(lambda: defaultdict(list))
comp_direction = {}  # 每个 competition 的方向，只需要记录一次

for exp, comp, score, direction in data:
    comp_base = strip_version(comp)
    comp2scores_raw[comp_base][exp].append(score)
    comp_direction[comp_base] = direction

# 第二步：求平均值
comp2scores = defaultdict(dict)
for comp, exp_dict in comp2scores_raw.items():
    direction = comp_direction[comp]   # 1 or -1
    for exp, score_list in exp_dict.items():
        comp2scores[comp][exp] = (np.mean(score_list),np.std(score_list))
        # if direction == 1:
        #     comp2scores[comp][exp] = np.mean(score_list)
        # else:
        #     comp2scores[comp][exp] = np.mean(score_list)

# target = "curious-cattle"

# for comp, score_dict in comp2scores.items():

#     if target not in score_dict:
#         continue

#     direction = comp_direction[comp]  #
#     base = score_dict[target]

    # diffs = {exp: (score - base) * direction for exp, score in score_dict.items()}

    # sorted_exps = sorted(diffs, key=diffs.get, reverse=True)
    # sorted_vals = [diffs[e] for e in sorted_exps]

    # plt.figure(figsize=(5, 4))
    # plt.barh(sorted_exps, sorted_vals)
    # plt.axvline(0, color='black', linewidth=1)
    # plt.title(f"Difference from humble-cardinal — {comp}")
    # plt.xlabel("Performance Difference (positive = better)")
    # plt.tight_layout()
    # plt.show()
    # plt.savefig(f'diff_{comp}.png')


In [ ]:
import pandas as pd

rows = []
for comp, exp_dict in comp2scores.items():
    for exp, (mean, std) in exp_dict.items():
        rows.append({
            "competition": comp,
            "experiment": exp,
            "mean": mean,
            "std": std,
        })

df_base = pd.DataFrame(rows)
print(df_base.head())


In [ ]:
df_base

In [ ]:
import os
import json
import pandas as pd


def build_sota_df(run_dirs, evo_list, experiment_name="rd-agent-sota"):
    rows = []

    for run_dir in run_dirs:
        run_name = os.path.basename(run_dir)

        json_files = [f for f in os.listdir(run_dir) if f.endswith(".json")]
        if not json_files:
            print(f"[WARN] No json found in {run_dir}")
            continue

        json_path = os.path.join(run_dir, json_files[0])
        print(f"Loading {json_path}")

        with open(json_path, "r") as f:
            data = json.load(f)

        for report in data.get("competition_reports", []):
            comp = report.get("competition_id")
            if comp is None:
                continue

            if comp in evo_list:
                rows.append({
                    "run": run_name,
                    "competition": comp,
                    "score": report.get("score"),
                })

    df = pd.DataFrame(rows)

    if df.empty:
        raise RuntimeError("No EVO competitions found.")

    # ✅ 这里已经是 3 次 run 的 mean / std
    df_sota = (
        df.groupby("competition")["score"]
          .agg(mean="mean", std="std")
          .reset_index()
    )

    df_sota["experiment"] = experiment_name
    return df_sota


RUN_DIRS1 = [ "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/rdagent_group9",
 "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/rdagent_group10",
   "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/rdagent_group11", ]



df_sota1 = build_sota_df(
    RUN_DIRS1,
    EVO,
    experiment_name="rd-agent-sota"
)

RUN_DIRS2 = [ "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/aira-dojo_group18",
 "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/aira-dojo_group19",
   "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/aira-dojo_group20", ]

# df_sota2 = build_sota_df(
#     RUN_DIRS2,
#     EVO,
#     experiment_name="aira-dojo"
# )

# RUN_DIRS3 = [ "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/internagent_group1",
#  "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/internagent_group2",
#    "/data/userdata/v-lijingyuan/mle_score/mle-bench/runs/internagent_group3", ]

# df_sota3 = build_sota_df(
#     RUN_DIRS3,
#     EVO,
#     experiment_name="Intern-agent"
# )


In [ ]:
df_1 = pd.concat(
    [df_base, df_sota1],#,df_sota2,df_sota3],
    ignore_index=True
)

In [ ]:
name_map = {
    "smooth-tuna": "RD-Agent-GPT5-baseline",
    "quality-cougar": "RD-Agent-GPT5-world-model",
    "rd-agent-sota":"RD-Agent-GPT5-SOTA",
    # "aira-dojo":"Aira-dojo"
}
df_1["experiment"] = df_1["experiment"].replace(name_map)


In [ ]:
df_1 = df_1.sort_values(by="competition").reset_index(drop=True)


In [ ]:
df_1.to_csv("final.csv")

In [ ]:
import pandas as pd
import numpy as np

def add_rank(g):
    # 同一个 competition 里 direction 是一致的
    direction = g["direction"].iloc[0]

    ascending = True if direction == "↓" else False

    g = g.sort_values("mean", ascending=ascending)
    g["rank"] = np.arange(1, len(g) + 1)
    return g

df_ranked = (
    df_1
    .groupby("competition", group_keys=False)
    .apply(add_rank)
)
top_stats = (
    df_ranked
    .assign(
        top1=lambda x: x["rank"] == 1,
        top2=lambda x: x["rank"] <= 2,
    )
    .groupby("experiment")[["top1", "top2"]]
    .sum()
    .reset_index()
)


In [ ]:
top_stats

In [ ]:
lower_is_better = {
    "leaf-classification",
    "spooky-author-identification",
}
target_experiments = {
    "RD-Agent-GPT5-world-model",
    "RD-Agent-GPT5-baseline",
    "RD-Agent-GPT5-SOTA",
}

df = df_1[df_1["experiment"].isin(target_experiments)].copy()
def pick_top2(g):
    comp = g["competition"].iloc[0]
    ascending = comp in lower_is_better  # ↓ 任务

    g = g.sort_values("mean", ascending=ascending)

    g["rank"] = [1, 2, 3]  # 固定只有 3 个
    return g.iloc[:2]      # 只保留 top1 / top2
df_top = (
    df.groupby("competition", group_keys=False)
      .apply(pick_top2)
)

summary = (
    df_top.groupby(["experiment", "rank"])
          .size()
          .unstack(fill_value=0)
          .rename(columns={1: "top1", 2: "top2"})
          .reset_index()
)

print(summary)


In [ ]:
summary

In [ ]:
def df_to_latex_table(df, higher_is_better=True, digits=4):
    lines = []

    lines.append(r"\begin{table}[t]")
    lines.append(r"\centering")
    lines.append(r"\resizebox{\linewidth}{!}{")
    lines.append(r"\begin{tabular}{l l c}")
    lines.append(r"\hline")
    lines.append(r"\textbf{Competition ID} (MLE-Bench) & \textbf{Method} & \textbf{Score$\pm$std} \\")
    lines.append(r"\hline")

    for comp, g in df.groupby("competition", sort=True):
        g = g.copy()

        # 找 best
        if higher_is_better:
            best_idx = g["mean"].idxmax()
        else:
            best_idx = g["mean"].idxmin()

        first = True
        for idx, row in g.iterrows():
            mean = row["mean"]
            std = row["std"]

            score_str = f"{mean:.{digits}f} \\pm {std:.{digits}f}"
            if idx == best_idx:
                score_str = rf"\textbf{{{mean:.{digits}f}}} \pm {std:.{digits}f}"

            if first:
                lines.append(
                    rf"{comp} ($\uparrow$) & {row['experiment']} & ${score_str}$ \\"
                )
                first = False
            else:
                lines.append(
                    rf" & {row['experiment']} & ${score_str}$ \\"
                )

        lines.append(r"\hline")

    lines.append(r"\end{tabular}")
    lines.append(r"}")
    lines.append(r"\end{table}")

    return "\n".join(lines)


In [ ]:
latex_str = df_to_latex_table(df_1)
print(latex_str)
